Train and predict with single step iteration. For multiple iterated predictions perform single step prediction looping back to predict subsequent step. Loss (and Accuracy) need to be updated for not directed prediction, but forward iteration of prediction since this is a many-to-one problem.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
from PIL import Image
import time
import math
from tqdm.auto import tqdm

In [ ]:
import sys
print(sys.version)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
SIZE = 25

**Rules for updating**

In [ ]:
# from http://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/
def life_step_1(X):
    """Game of life step using generator expressions"""
    nbrs_count = sum(np.roll(np.roll(X, i, 0), j, 1)
                     for i in (-1, 0, 1) for j in (-1, 0, 1)
                     if (i != 0 or j != 0))
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

In [ ]:
def draw_image(img):
    img = Image.fromarray(np.uint8(img) * 255)
    return img

In [ ]:
def plot_animate(arr):
    clear_output(wait=True)
    plt.imshow(draw_image(arr), cmap='gray')
    plt.show()

In [ ]:
# WORKING WITH THE DATA
# parts based on https://www.kaggle.com/candaceng/understanding-the-problem-and-eda

**Load data**

In [ ]:
train_df = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/train.csv')
test_df = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/test.csv')
print(train_df.shape)
print(test_df.shape)

See count distribution for number of iteration steps. They are fairly uniformly distributed

In [ ]:
train_df.groupby(['delta']).size()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
num_bins = 50

#plot final cell distr after time steps
for delta in range(1,6):
    mask = train_df.loc[train_df['delta'] == delta]
    counts = mask.iloc[:, 627:][mask.iloc[:, 627:] == 1].count(axis=1).values
    ax.hist(counts, num_bins, density=True, label=f'{delta}', alpha=0.5)

ax.set_xlabel('Number of Alive Cells')
ax.set_ylabel('Probability density')
ax.legend(prop={'size': 10})    
fig.tight_layout()
plt.show()

In [ ]:
# SELECTING ONE SAMPLE TO VISUALIZE

In [ ]:
train_sample = train_df.sample()

In [ ]:
sample_start = train_sample.loc[:, train_sample.columns.str.startswith('start')]
sample_stop = train_sample.loc[:, train_sample.columns.str.startswith('stop')]

In [ ]:
start_arr = np.asarray(sample_start).reshape(25, 25)
stop_arr = np.asarray(sample_stop).reshape(25, 25)
# time step 
time_step = train_sample['delta'].values[0]
print(time_step)

In [ ]:
def plot_comp_step(arr1, arr2, step):
    fig, ax = plt.subplots(1,2, figsize=(12,12))
    ax[0].imshow(draw_image(arr1), cmap='gray')
    ax[0].set_title('start')
    ax[0].axis('off')
    ax[1].imshow(draw_image(arr2), cmap='gray')
    ax[1].set_title(f'stop after: {step}')
    ax[1].axis('off')
    plt.show()

In [ ]:
plot_comp_step(start_arr, stop_arr, time_step)

**Display as individual frames**

In [ ]:
updated_arr = np.copy(start_arr)
steps = []
steps.append(updated_arr)
for x in range(time_step):
    updated_arr = life_step_1(updated_arr)
    steps.append(updated_arr)

In [ ]:
fig, m_axs = plt.subplots(1, len(steps), figsize = (10,20))
for c_ax, c_row in zip(m_axs.flatten(), steps):
    c_ax.imshow(c_row, cmap='gray')
    c_ax.axis('off')

# Consider only single step for now

In [ ]:
# CREATING SINGLE STEP DATASET

In [ ]:
single_step_df = train_df.loc[train_df['delta'] == 1]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
num_bins = 50

delta = 1
counts = single_step_df.iloc[:, 2:627][single_step_df.iloc[:, 2:627] == 1].count(axis=1).values
ax.hist(counts, num_bins, density=True, label=f'start', alpha=0.5)
counts = single_step_df.iloc[:, 627:][single_step_df.iloc[:, 627:] == 1].count(axis=1).values
ax.hist(counts, num_bins, density=True, label=f'stop', alpha=0.5)

ax.set_xlabel('Number of Alive Cells')
ax.set_ylabel('Probability density')
ax.legend(prop={'size': 10})    
fig.tight_layout()
plt.show()

In [ ]:
fig, m_axs = plt.subplots(5, 2, figsize=(12,12))
for i, (c_ax, c_row) in enumerate(zip(m_axs.flatten(), single_step_df.sample(5).iterrows())):
    
    m_axs[i,0].imshow(np.asarray(c_row[1][627:]).reshape(25,25).astype('uint8'))
    m_axs[i,0].set_title(c_row[0])
    m_axs[i,0].axis('off')
    
    m_axs[i,1].imshow(np.asarray(c_row[1][2:627]).reshape(25,25).astype('uint8'))
    m_axs[i,1].set_title(c_row[0])
    m_axs[i,1].axis('off')

Resize to simulate periodic boundary. Check on single slice

In [ ]:
idx = 13970
single_img = np.asarray(single_step_df.loc[idx][2:627]).reshape(25,25).astype('uint8')

In [ ]:
single_img_33 = np.pad(single_img, (3,3), 'wrap')
single_img_33 = np.pad(single_img_33, (1,1), constant_values=(0,0))
#single_img_32 = single_img_33[:-1, :-1]
single_img_33.shape

In [ ]:
fig, ax = plt.subplots()
plt.imshow(single_img_33)
plt.show()

In [ ]:
end_single_img_33 = life_step_1(single_img_33)

In [ ]:
single_img_33[4:-4, 4:-4].shape

In [ ]:
fig, ax = plt.subplots()
plt.imshow(end_single_img_33[4:-4, 4:-4])
plt.show()

In [ ]:
def plot_comp(arr1, arr2, *args, labels=['']):
    lst=[]
    lst.append(arr1)
    lst.append(arr2)
    for arg in args:
        lst.append(arg)
    n = len(lst)
    if labels == ['']:
        labels = labels * n
    fig, ax = plt.subplots(1,n, figsize=(12,12))
    for idx in range(n):
        ax[idx].imshow(draw_image(lst[idx]), cmap='gray')
        ax[idx].set_title(labels[idx])
        ax[idx].axis('off')

    plt.show()

In [ ]:
plot_comp(end_single_img_33[4:-4, 4:-4], np.asarray(single_step_df.loc[idx][627:]).reshape(25,25).astype('uint8'), labels = ['end process','end orig'])

Above, we return the exact same result without periodic boundary required. 

In [ ]:
def preprocess(arr): #resize to 33x33
    arr = np.pad(arr, (3,3), 'wrap')
    arr = np.pad(arr, (1,1), constant_values=(0,0))
    return arr#[:-1, :-1]

In [ ]:
preprocess(single_img).shape

In [ ]:
def postprocess(arr): # returns shape (25, 25) from shape (33,33). same for tensor, check shapes though
    return arr[4:-4, 4:-4]

In [ ]:
start_key = ['start_' + str(i) for i in range(625)]
stop_key = ['stop_' + str(i) for i in range(625)]

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, Conv2DTranspose, \
    concatenate, Dropout, Lambda, MaxPooling2D, BatchNormalization, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.data import Dataset
from tensorflow.keras import backend as K

In [ ]:
K.image_data_format()

In [ ]:
# need to use tensor not numpy
# https://stackoverflow.com/questions/49192051/converting-tensor-to-np-array-using-k-eval-in-keras-returns-invalidargumenterr

# Simulate more single step data

In [ ]:
probs = [[0.90, 0.10],
         [0.80, 0.20],
         [0.70, 0.30],
         [0.60, 0.40],
         [0.50, 0.50],
         [0.40, 0.60],
         [0.30, 0.70],
         [0.20, 0.80],
         [0.10, 0.90]]

In [ ]:
single_step_df.shape # current amount

In [ ]:
new_single_step_df = pd.DataFrame(columns=list(train_df.columns), dtype=np.int64)

In [ ]:
%%time
for i in range(10000):
    arr = np.random.choice([0,1], (SIZE, SIZE), p=probs[np.random.choice(len(probs))])#probs[np.random.choice(len(probs))]
    #warm-up, based on Kaggle desccription
    for j in range(5):
        arr = life_step_1(arr)
    # 1 interation
    update_arr = life_step_1(arr)

    new_row = np.concatenate((np.array([len(single_step_df) + int(i+1)]).reshape(1,-1), np.array([1]).reshape(1,-1), arr.reshape(-1, 625).round(0).astype('uint8'), update_arr.reshape(-1, 625).round(0).astype('uint8')), axis=1)
    new_single_step_df = new_single_step_df.append(pd.DataFrame(new_row, columns=list(train_df.columns)), ignore_index=True)

In [ ]:
new_single_step_df = single_step_df.append(new_single_step_df, ignore_index=True)

In [ ]:
new_single_step_df.shape

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
num_bins = 50

counts = single_step_df.iloc[:, 2:][single_step_df.iloc[:, 2:] == 1].count(axis=1).values
ax.hist(counts, num_bins, density=True, label=f'original', alpha=0.5)
counts = new_single_step_df.iloc[:, 2:][new_single_step_df.iloc[:, 2:] == 1].count(axis=1).values
ax.hist(counts, num_bins, density=True, label=f'new', alpha=0.5)

ax.set_xlabel('Number of Alive Cells')
ax.set_ylabel('Probability density')
ax.legend(prop={'size': 10})    
fig.tight_layout()
plt.show()

Get any 0 images

In [ ]:
counts = single_step_df.iloc[:, 2:][single_step_df.iloc[:, 2:] == 1].count(axis=1).values

In [ ]:
fig, m_axs = plt.subplots(5, 2, figsize=(12,12))
for i, (c_ax, c_row) in enumerate(zip(m_axs.flatten(), new_single_step_df.sample(5).iterrows())):
    
    m_axs[i,0].imshow(np.asarray(c_row[1][627:]).reshape(25,25).astype('uint8'))
    m_axs[i,0].set_title(c_row[0])
    m_axs[i,0].axis('off')
    
    m_axs[i,1].imshow(np.asarray(c_row[1][2:627]).reshape(25,25).astype('uint8'))
    m_axs[i,1].set_title(c_row[0])
    m_axs[i,1].axis('off')

# Training

In [ ]:
single_step_df = new_single_step_df.copy()

In [ ]:
trainY = single_step_df.loc[:, single_step_df.columns.str.startswith('start')].values.reshape(-1, 25, 25)
trainX = single_step_df.loc[:, single_step_df.columns.str.startswith('stop')].values.reshape(-1, 25, 25)

In [ ]:
trainX.shape

In [ ]:
trainX = np.array([preprocess(xi) for xi in trainX]).astype(np.float32)
trainX = np.expand_dims(trainX, axis=-1).astype(np.float32)

In [ ]:
trainX.shape

In [ ]:
trainY = np.array([preprocess(xi) for xi in trainY]).astype(np.float32)
trainY = np.expand_dims(trainY, axis=-1).astype(np.float32)

In [ ]:
trainY.shape

In [ ]:
def life_step_1_tensor(X):
    """Game of life step using generator expressions"""
    nbrs_count = tf.stack([tf.roll(tf.roll(X, i, 0), j, 1)
                     for i in (-1, 0, 1) for j in (-1, 0, 1)
                     if (i != 0 or j != 0)])
    nbrs_count = tf.squeeze(K.sum(nbrs_count, axis=0, keepdims=True), axis=0)
    nbrs_count = (nbrs_count == 3) | ((X == 1) & (nbrs_count == 2))
    return tf.cast(nbrs_count, dtype=tf.uint32)

In [ ]:
def postprocess_tensor(arr): # returns shape (25, 25) from shape (31,31). same for tensor, check shapes though
    return arr[4:-4, 4:-4,:]

In [ ]:
def wrap_pad(t, extra_dims):
    s = tf.shape(t)
    m = tf.constant([extra_dims[0], extra_dims[1]])
    d = tf.constant([1, 3, 3, s.numpy()[-1]])
    t = tf.tile(t, d)[:, s[1]-m[0]:m[0]-s[1], s[2]-m[1]:m[1]-s[2], :]
    paddings = tf.constant([[0,0],[1, 1],[1, 1], [0,0]])
    t = tf.pad(t, paddings, 'CONSTANT')
    return t

In [ ]:
def life_step_1_tensor_sum(X):
    """Game of life step using generator expressions"""
    nbrs_count = tf.stack([tf.roll(tf.roll(X, i, 0), j, 1)
                     for i in (-1, 0, 1) for j in (-1, 0, 1)
                     if (i != 0 or j != 0)])
    nbrs_count = tf.squeeze(K.sum(nbrs_count, axis=0, keepdims=True), axis=0)
    return tf.cast(nbrs_count, dtype=tf.float16)

In [ ]:
t = trainY[0, :, :, :]

In [ ]:
t1 = life_step_1_tensor(t)
print(t1.shape)

In [ ]:
plot_comp(postprocess_tensor(t1)[:,:,0], postprocess_tensor(trainX[0, :, :])[:,:,0], labels = ['end process','end target'])

In [ ]:
t_sum = life_step_1_tensor_sum(t)
fig, axs = plt.subplots()
plt.imshow(postprocess_tensor(t_sum)[:,:,0].numpy().astype(np.uint16))
plt.show()

In [ ]:
def custom_loss(y_actual,y_pred):
    
#     pred_fwd = y_pred
#     actual_fwd = y_actual
    y_actual_f = tf.cast(K.flatten(y_actual), tf.float32)
    y_pred_f = tf.cast(K.flatten(y_pred), tf.float32)
    bce = BinaryCrossentropy(from_logits=False)
    bce(y_actual_f, y_pred_f)
    #focal_tversky_2d(y_actual, y_pred)
    return bce(y_actual_f, y_pred_f)

In [ ]:
def dice_coef_loss(y_true, y_pred):
    H, W, C = y_true.shape[1:]
    smooth = 1e-5
    pred_flat = tf.reshape(y_pred, [-1, H * W * C])
    true_flat = tf.reshape(y_true, [-1, H * W * C])
    intersection = 2 * tf.reduce_sum(pred_flat * true_flat, axis=1) + smooth
    denominator = tf.reduce_sum(pred_flat, axis=1) + tf.reduce_sum(true_flat, axis=1) + smooth
    loss = 1 - tf.reduce_mean(intersection / denominator)
    return loss

In [ ]:
def focal_tversky_2d(y_true, y_pred, alpha=0.7, gamma=0.75):
    H, W, C = y_true.shape[1:]
    smooth = 1e-5
    y_pred_pos = tf.reshape(y_pred, [-1, H * W * C])
    y_true_pos = tf.reshape(y_true, [-1, H * W * C])
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos, axis=1)
    false_neg = tf.reduce_sum(y_true_pos * (1 - y_pred_pos), axis=1)
    false_pos = tf.reduce_sum((1 - y_true_pos) * y_pred_pos, axis=1)
    tversky = (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)
    loss = 1 - tf.reduce_mean(tversky)
    loss = tf.pow(loss, gamma)
    return loss

In [ ]:
def crglnet_v1():

    inputShape = (33, 33, 1)

    inputs = Input(inputShape)
    
#     inputs = Lambda(lambda x: wrap_pad(x, (3,3)))(inputs)# to change shape of input
    
    c1 = Conv2D(64, (3, 3), activation='elu', padding='same')(inputs)
    
    c2 = Conv2D(64, (3, 3), activation='elu', padding='same')(c1)

    c3 = Conv2D(128, (3, 3), activation='elu', padding='same')(c2)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same')(c3)
    
    c5 = Conv2D(64, (3, 3), activation='elu', padding='same')(c4)

    c6 = Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = Model(inputs=[inputs], outputs=[c6])
    
    return model


In [ ]:
def crglnet_v2():

    inputShape = (33, 33, 1)

    inputs = Input(inputShape)
    
#     inputs = Lambda(lambda x: wrap_pad(x, (3,3)))(inputs)# to change shape of input
    
    c1 = Conv2D(32, (3, 3), activation='elu', padding='same')(inputs)
    
    c2 = Conv2D(64, (3, 3), activation='elu', padding='same')(c1)

    c3 = Conv2D(128, (5, 5), activation='elu', padding='same')(c2)
    
    c4 = concatenate([Conv2D(64, (3, 3), activation='elu', padding='same')(c3), c2])
    
    c5 = concatenate([Conv2D(32, (3, 3), activation='elu', padding='same')(c4), c1])

    c6 = Conv2D(64, (3, 3), activation='elu', padding='same')(c5)
    
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same')(c6)

    c8 = Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs=[inputs], outputs=[c8])
    
    return model

In [ ]:
def crglnet_v3():

    inputShape = (33, 33, 1)

    inputs = Input(inputShape)
    
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)

    c3 = Conv2D(128, (5, 5), activation='relu', padding='same')(c2)
    
    c4 = Conv2D(128, (5, 5), activation='relu', padding='same')(c3)
    
    c5 = concatenate([Conv2D(32, (3, 3), activation='relu', padding='same')(c4), c1])

    c6 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)
    
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same')(c6)

    c8 = Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs=[inputs], outputs=[c8])
    
    return model

In [ ]:
def crglnet_v4():

    image_inputShape = (33, 33, 1)
    #feat_image_inputShape = (33, 33, 1)
    
    image_inputs = Input(image_inputShape)
    feature_inputs = Lambda(lambda x: life_step_1_tensor_sum(x))(image_inputs)
    
    bn1 = BatchNormalization()(image_inputs)
    ci1 = Conv2D(32, (3, 3), activation='elu', padding='same')(bn1)
    
    bn2 = BatchNormalization()(feature_inputs)  
    cf1 = Conv2D(32, (3, 3), activation='elu', padding='same')(bn2)
    
    concat_layer= concatenate([image_inputs, feature_inputs])
#     bn3 = BatchNormalization()(concat_layer) 
    
#     inputs = Lambda(lambda x: wrap_pad(x, (3,3)))(inputs)# to change shape of input
    
    c1 = Conv2D(32, (3, 3), activation='elu', padding='same')(concat_layer)
    
    c2 = Conv2D(64, (3, 3), activation='elu', padding='same')(c1)

    c3 = Conv2D(128, (5, 5), activation='elu', padding='same')(c2)
    
    c4 = concatenate([Conv2D(64, (3, 3), activation='elu', padding='same')(c3), c2])
    
    c5 = concatenate([Conv2D(32, (3, 3), activation='elu', padding='same')(c4), c1])

    c6 = Conv2D(64, (3, 3), activation='elu', padding='same')(c5)
    
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same')(c6)

    c8 = Conv2D(1, (1, 1), activation='sigmoid')(c7)

    model = Model(inputs=[image_inputs], outputs=[c8])
    
    return model

In [ ]:
model = crglnet_v4()

In [ ]:
opt = Adam(lr=1e-3)
model.compile(optimizer=opt, loss=custom_loss, metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=2)])

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

In [ ]:
#https://stackoverflow.com/questions/58947679/no-gradients-provided-for-any-variable-in-tensorflow2-0
def step(train_x, true_y):
    loss = []
    with tf.GradientTape() as tape:

        # Make prediction
        pred_y = model(train_x)
        # Calculate loss
        # sends a batch
        pred_y = tf.cast(pred_y,tf.float32)
        # This is incorrect - cannot compute gradient with binary rule. Needs probs
#         thresh = tf.constant([0.5], dtype=tf.float32)
#         pred_fwd = tf.cast(tf.where(pred_y>thresh,tf.constant(1),tf.constant(0)), tf.uint32)
#         pred_fwd = tf.map_fn(life_step_1_tensor, pred_fwd, fn_output_signature=tf.uint32)
#         true_fwd = tf.map_fn(life_step_1_tensor, true_y, fn_output_signature=tf.uint32)
#         test_loss = custom_loss(true_fwd, pred_fwd)
        model_loss = custom_loss(true_y, pred_y)# + test_loss
        loss.append(model_loss.numpy())
        
    
    # Calculate gradients
    model_gradients = tape.gradient(model_loss, model.trainable_variables)
    # Update model
    opt.apply_gradients(zip(model_gradients, model.trainable_variables))
    return np.mean(loss)

In [ ]:
# Training loop
epochs = 20
batch_size = 32
bat_per_epoch = math.floor(len(train_x) / batch_size)
epoch_loss = []
val_acc = []
for epoch in tqdm(range(epochs)):
    step_loss = []
    for i in tqdm(range(bat_per_epoch)):
        n = i*batch_size
        step_loss.append(step(train_x[n:n+batch_size], train_y[n:n+batch_size]))
        
    epoch_loss.append(np.mean(step_loss))
    val_acc.append(model.evaluate(test_x, test_y, verbose=0)[1])
        
    print(f'Epoch: {epoch}, Loss: {epoch_loss[epoch]}, Val_acc: {val_acc[epoch]}')
#model.save('model_iterative_1step_40epochs')

In [ ]:
# Calculate accuracy
model.compile(optimizer=opt, loss=custom_loss, metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=2)]) # Compile just for evaluation
print(model.evaluate(test_x, test_y, verbose=0))

In [ ]:
preds = model.predict(test_x)
preds.shape

In [ ]:
preds_thresh = tf.where(preds>0.5,1,0)
pred_fwd = tf.map_fn(life_step_1_tensor, preds_thresh, fn_output_signature=tf.uint32)

In [ ]:
actual_fwd = tf.map_fn(life_step_1_tensor, test_y, fn_output_signature=tf.uint32)

In [ ]:
idx = 1001
labels = ['given end','pred start', 'end process','end target']
plot_comp(postprocess_tensor(test_x[idx, :, :, :])[:, :, 0], postprocess_tensor(preds_thresh[idx, :, :, :])[:, :, 0], postprocess_tensor(pred_fwd[idx,:,:,:])[:,:,0], postprocess_tensor(actual_fwd[idx,:,:,:])[:,:,0], labels = labels)

In [ ]:
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.plot(np.arange(0, epochs), epoch_loss, label="custom_loss", color=color)
ax1.set_ylabel("Loss", color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.plot(np.arange(0, epochs), val_acc, label="val_acc")
ax2.set_ylabel("Val_Acc", color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.legend()
plt.show()

# MAKE PREDICTIONS ITERATIVELY

In [ ]:
start_features = [f for f in train_df.columns if "start" in f]
stop_features = [f for f in train_df.columns if "stop" in f]

In [ ]:
sub = pd.DataFrame()
m = tf.keras.metrics.Accuracy()
accuracies = []
t1 = tqdm(range(1,6), desc=f'Delta: ')
for delta in t1:
    m.reset_states()
    
    t1.set_description(f'Delta: {delta}')
    t1.refresh()
    
    test_data_iter = train_df.loc[train_df['delta'] == delta]
    tmp_sub = test_data_iter[["id"]].copy()
    
    testY = test_data_iter.loc[:, test_data_iter.columns.str.startswith('start')].values.reshape(-1, 25, 25)
    testX = test_data_iter.loc[:, test_data_iter.columns.str.startswith('stop')].values.reshape(-1, 25, 25)
    
    testX = np.array([preprocess(xi) for xi in testX]).astype(np.float32)
    testX = np.expand_dims(testX, axis=-1).astype(np.float32)
    
    testY = np.array([preprocess(xi) for xi in testY]).astype(np.float32)
    testY = np.expand_dims(testY, axis=-1).astype(np.float32)
    
    t2 = tqdm(range(delta))
    for i in t2:
        if i == 0:
            preds = model.predict(testX)
        else:
            preds = tf.where(preds>0.5,1,0)
            preds = model.predict(preds)
            
    preds = tf.cast(tf.where(preds>0.5,1,0), tf.uint32)
    m.update_state(preds, testY)
    acc = m.result().numpy()
    print(f'Accuracy: {acc}')
    accuracies.append(acc) 
    
    preds = preds[:, 4:-4, 4:-4,:].numpy()
    tmp = pd.DataFrame(preds.reshape(-1, 625).astype(np.uint8), columns=start_features, index=tmp_sub['id'])
    tmp_sub = tmp_sub.join(tmp)
    sub = sub.append(tmp_sub)
sub.sort_index(inplace = True)

In [ ]:
print(f'Mean accuracy: {np.array(accuracies).mean()}')
print(f'LB score estimate from training: {1 - np.array(accuracies).mean()}')

In [ ]:
# TEST DATA PREDICTION FOR SUBMISSION
sub = pd.DataFrame()
m = tf.keras.metrics.Accuracy()
accuracies = []
t1 = tqdm(range(1,6), desc=f'Delta: ')
for delta in t1:
    m.reset_states()
    
    t1.set_description(f'Delta: {delta}')
    t1.refresh()
    
    test_data_iter = test_df.loc[test_df['delta'] == delta]
    tmp_sub = test_data_iter[["id"]].copy()
    tmp_sub.set_index(tmp_sub['id'].values)
    
    testX = test_data_iter.loc[:, test_data_iter.columns.str.startswith('stop')].values.reshape(-1, 25, 25)
    
    testX = np.array([preprocess(xi) for xi in testX]).astype(np.float32)
    testX = np.expand_dims(testX, axis=-1).astype(np.float32)
    
    t2 = tqdm(range(delta))
    for i in t2:
        if i == 0:
            preds = model.predict(testX)
        else:
            preds = tf.where(preds>0.5,1,0)
            preds = model.predict(preds)
            
    preds = tf.cast(tf.where(preds>0.5,1,0), tf.uint32)   
    preds = preds[:, 4:-4, 4:-4,:].numpy()
    tmp = pd.DataFrame(preds.reshape(-1, 625).astype(np.uint8), columns=start_features, index=tmp_sub['id'].values)
    tmp.insert(loc = 0, column='id', value=tmp_sub['id'].values)
    sub = sub.append(tmp)
sub.sort_index(inplace = True)
sub.reset_index(drop = True, inplace = True)

In [ ]:
sub.to_csv('submission.csv', index=False)